In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path

#PERSONAL PARAMETERS YOU CAN OBTAIN FROM YOUR REDIS ACCOUNT
REDIS_HOST = "## ADD YOU REDIS HOST HERE ##"
REDIS_PORT = 00000   #ADD YOUR REDIS PORT HERE
REDIS_USERNAME = "## ADD YOUR REDIS USERNAME HERE ##"
REDIS_PASSWORD = "## ADD YOUR REDIS PASSWORD HERE ##"

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)


# endpoint /status
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

# /devices
class Devices(object):
    exposed = True #this makes the endpoint available for the users
    
    def GET(self, *path, **query): #path and query are used in case you have them in the http request path is a list, query is a dict
        keys = redis_client.keys('*:battery')
        items = []

        for key in keys:
            key = key.decode()
            uid = key.removesuffix(':battery')
            items.append(uid)

        response_dict = {
            'mac_addresses' : items
        }
        
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        
        response = json.dumps(response_dict)
        return response


# /device/{mac_address}
class SingleDevice(object):
    '''Retrieve battery status information of the device with the specified MAC address
    in the specified time range.'''
    exposed = True
    def GET(self, *path, **query):
        ''' 
        o 200 – OK: Everything worked as expected.
        o 400 – Bad Request: missing MAC address.
        o 400 – Bad Request: missing start time.
        o 400 – Bad Request: missing end time.
        o 404 – Not Found: invalid MAC address.
        '''

        #--------------------------------------------------Error handling-----------------------------------------------------------
        
        # missing MAC address
        if(len(path)) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')  #compliant to the documentation
        
        # missing start time 
        query_from = query.get('from', None)
        if query_from is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')  #compliant to the documentation
                
        # missing end time
        query_to = query.get('to', None)
        if query_to is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')  #compliant to the documentation
        
        # invalid MAC address
        keys = redis_client.keys('*:battery')
        macs = []
        for key in keys:
            key = key.decode()
            uid = key.removesuffix(':battery')
            macs.append(uid)
        mac_address_from_path = path[0] #path contains something (checked before), now I check if it exists in Redis
        if mac_address_from_path not in macs:
            raise cherrypy.HTTPError(404, '404 – Not Found: invalid MAC address.') #compliant to the documentation
        
        #--------------------------------------------------end of Error handling-----------------------------------------------------------

        #finding all the data needed
        #[(1666104016831, 25.0), (1666104017012, 25.0), (1666104017184, 25.0)] this is the format returned by RANGE()

        print('mac address is    ' + mac_address_from_path)
        
        item = redis_client.ts().get(f'{mac_address_from_path}:battery')
        print('item is ')
        print(item)
        print()

        battery_val = redis_client.ts().range(f'{mac_address_from_path}:battery',query_from, query_to) 
        power_val = redis_client.ts().range(f'{mac_address_from_path}:power',query_from, query_to)
        
        
        print('battery val is ')
        print(battery_val)
        print('battery vals type is ')
        print(type(battery_val))
        print()
        
        # * operator is used to unpack these inner lists so that they can be passed as separate arguments to the zip function.
        timestamps_battery, battery_values = zip(*battery_val) 
        timestamps_power, power_values = zip(*power_val)
        
        print()
        print('timestamps_battery')
        print(timestamps_battery)
        print('battery_values')
        print(battery_values)
        print('power_values')
        print(power_values)
        print()

        #building the response_dictionary
        response_dict = {
            'mac_address': mac_address_from_path,
            'timestamps' : timestamps_battery,
            'battery_levels': battery_values,
            'power_plugged': power_values
            }
        print(response_dict)

        #If everything worked fine
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        
        response = json.dumps(response_dict)
        return response
    

    ''': Delete the timeseries associated to the specified MAC address.'''
    def DELETE(self, *path, **query):
        '''
            o 200 – OK: Everything worked as expected.
            o 400 – Bad Request: missing MAC address.
            o 404 – Not Found: invalid MAC address.
        '''
        # missing MAC address
        if(len(path)) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')  #compliant to the documentation

        # invalid MAC address
        keys = redis_client.keys('*:battery')
        macs = []
        for key in keys:
            key = key.decode()
            uid = key.removesuffix(':battery')
            macs.append(uid)
        mac_address_from_path = path[0] #path contains something (checked before), now I check if it exists in Redis
        if mac_address_from_path not in macs:
            raise cherrypy.HTTPError(404, '404 – Not Found: invalid MAC address.') #compliant to the documentation
        
        #Deleting the timeseries
        redis_client.delete(f'{mac_address_from_path}:battery')
        redis_client.delete(f'{mac_address_from_path}:power')

        #If everything worked fine
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        return




if __name__ == '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
        }

    #It's important to map each class to the correct endpoint later
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(SingleDevice(), '/device', conf)
        
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[14/Jan/2023:08:37:40] ENGINE Bus STARTING
[14/Jan/2023:08:37:40] ENGINE Started monitor thread 'Autoreloader'.
[14/Jan/2023:08:37:40] ENGINE Serving on http://0.0.0.0:8080
[14/Jan/2023:08:37:40] ENGINE Bus STARTED
Redis Connected: True


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6729a98-a1be-4c88-8020-81eb03e99821' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>